In [ ]:
import json
from pathlib import Path
from os.path import join as path_join

repos_lines = !wget -qO - https://api.github.com/orgs/spoonlabs/repos
repos = json.loads("\n".join(repos_lines))
repos
for repo in repos:
    dest = path_join("repos", repo["name"])
    if not Path(dest).exists():
        !git clone {repo["ssh_url"]} {dest}

In [ ]:
%matplotlib inline

import logging
import coloredlogs
import matplotlib as mpl
import matplotlib.pyplot as plt
from run import display_html, no_logs, run, run_one, run_and_print

coloredlogs.install(level="DEBUG")
logging.getLogger("matplotlib.axes._base").setLevel(logging.INFO)

In [ ]:
!srcd init repos

In [ ]:
java_file = "repos/spoon-examples/src/main/java/fr/inria/gforge/spoon/HelloWorldProcessor.java"
!cat {java_file}

In [ ]:
!srcd parse lang {java_file}

In [ ]:
!srcd parse uast {java_file}

In [ ]:
!srcd parse uast {java_file} --query "//uast:Identifier/Name"

In [ ]:
!srcd sql "SHOW TABLES;"

In [ ]:
run_and_print("""
SELECT repository_id, cf.file_path, LANGUAGE(cf.file_path, f.blob_content) as lang
FROM   files AS f
       NATURAL JOIN commit_files AS cf
       NATURAL JOIN refs
WHERE  ref_name = 'HEAD'
       AND lang = 'Python';
""")

In [ ]:
python_file = "repos/nopol-experiments/src/Config.py"


identifiers = !srcd parse uast {python_file} --query "//uast:Identifier/Name"


for identifier in sorted(set(i[1:-1] for i in identifiers[1:])):
    print(identifier)

In [ ]:
!srcd sql "SELECT * FROM repositories;"

In [ ]:
repos = run("""
    SELECT * FROM repositories;
""")
print("got %d repositories" % len(repos))

In [ ]:
commits = run("""
  SELECT commit_hash
  FROM ref_commits
  WHERE ref_name = 'HEAD'
    ORDER BY history_index DESC
""")
print('got %d commits on HEAD' % len(commits))

In [ ]:
# get the corresponding timestamps

timestamps = run("""
  SELECT commit_author_when
  FROM commits
  WHERE commit_hash IN (%s)
  ORDER BY commit_author_when
""" % (', '.join('"%s"' % x for x in commits)))

In [ ]:
years, months, days, counts = run("""
  SELECT YEAR(commit_author_when) as year,
         MONTH(commit_author_when) as month,
         DAY(commit_author_when) as day,
         COUNT(*) as n
  FROM commits
  GROUP BY year, month, day
  ORDER BY year, month, day
""")

In [ ]:
import datetime
from numpy import zeros
import pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy.signal import convolve, slepian

def scatter_counts(years, months, days, counts):
    """Turn a date-value mapping to a uniform time series."""
    dr = pandas.date_range(datetime.datetime(years[0], months[0], days[0]),
                           datetime.datetime(years[-1], months[-1], days[-1]))
    vals = zeros(len(dr))
    filled = {(y, m, d): c for (y, m, d, c) in zip(years, months, days, counts)}
    for i, dt in enumerate(dr):
        vals[i] = filled.get((dt.year, dt.month, dt.day), 0)
    window = slepian(32, 0.5)
    window /= sum(window)
    return dr, convolve(vals, window, "same")

def plot_commits_through_time(years, months, days, counts):
    plt.title("number of commits through time")
    plt.fill_between(*scatter_counts(years, months, days, counts))
    plt.grid(False)

plt.figure(figsize=(15, 3))
plot_commits_through_time(years, months, days, counts)



In [ ]:


repos, years, months, days, counts = run("""
  SELECT repository_id as repo,
         YEAR(commit_author_when) as year,
         MONTH(commit_author_when) as month,
         DAY(commit_author_when) as day,
         COUNT(*) as n
  FROM commits
  WHERE commit_author_when <= now()
  GROUP BY repo, year, month, day
  ORDER BY repo, year, month, day
""")

from itertools import groupby
repos_dict = {}
for r, g in groupby(zip(repos, years, months, days, counts), lambda t: t[0]):
    g = list(g)
    repos_dict[r] = [[e[i] for e in g] for i in range(1, 5)]
ncommits = sorted([(sum(p[-1]), r) for (r, p) in repos_dict.items()], reverse=True)
timeline = sorted(set(zip(years, months, days)))
date_range = pandas.date_range(datetime.datetime(*timeline[0]),
                               datetime.datetime(*timeline[-1]))

def smooth_scatter_counts(years, months, days, counts):
    """Turn a date-value mapping to a uniform time series."""
    vals = zeros(len(date_range))
    filled = {(y, m, d): c for (y, m, d, c) in zip(years, months, days, counts)}
    for i, dt in enumerate(date_range):
        vals[i] = filled.get((dt.year, dt.month, dt.day), 0)
    window = slepian(32, 0.5)
    window /= sum(window)
    return convolve(vals, window, "same")

plt.figure(figsize=(15, 3))

plt.title("number of commits through time")
matrix = zeros((len(repos_dict), len(date_range)))
for i, (_, repo) in enumerate(ncommits):
      matrix[i] = smooth_scatter_counts(*repos_dict[repo])
plt.stackplot(date_range, matrix, labels=[r for (_, r) in ncommits])
plt.xticks(rotation=60)
plt.legend(mode="expand", loc="lower center", ncol=6, bbox_to_anchor=(0, -1.6, 1., 0))
plt.show()

In [ ]:
langs, counts = run("""
SELECT lang, COUNT(*) as n
FROM (
    SELECT LANGUAGE(cf.file_path, f.blob_content) as lang
    FROM   files AS f
           NATURAL JOIN commit_files cf
           NATURAL JOIN refs
    WHERE  ref_name = 'HEAD'
           AND file_path NOT LIKE 'third_party/%%'
) AS t
WHERE lang is not null
GROUP BY lang
ORDER BY 2 DESC
""")

In [ ]:
percents = [(100 * c) / sum(counts) for c in counts if c / sum(counts) > 0.005]
if len(percents) < len(langs):
    percents.append(100 - sum(percents))
    labels = langs[:len(percents)-1] + ("others",)
else:
    labels = langs

plt.figure(figsize=(10, 10))

plt.title("# files per language")
_ = plt.pie(percents, labels=labels, textprops={"fontsize": 15})

In [ ]:
langs, counts = run("""
SELECT lang, SUM(n)
FROM (
    SELECT lang, SUM(lines) as n
    FROM (
        SELECT f.repository_id, LANGUAGE(cf.file_path, f.blob_content) as lang,
               ARRAY_LENGTH(SPLIT(f.blob_content, '\n')) as lines
        FROM   files AS f
               NATURAL JOIN commit_files cf
               NATURAL JOIN refs
        WHERE  ref_name = 'HEAD'
           AND file_path NOT LIKE 'third_party/%%'
    ) AS t
    WHERE lang IS NOT NULL
    GROUP BY repository_id, lang
) AS t
GROUP by lang
ORDER BY n DESC
""")

In [ ]:
percents = [(100 * c) / sum(counts) for c in counts if c / sum(counts) > 0.004]
if len(percents) < len(langs):
    percents.append(100 - sum(percents))
    langs = langs[:len(percents)-1] + ("others",)

plt.figure(figsize=(10, 10))

plt.title("# lines of code per language")
_ = plt.pie(percents, labels=langs, textprops={"fontsize": 15})



In [ ]:
orgs, counts = run("""
    SELECT JSON_EXTRACT(SPLIT(committer_email, '@'), '$[1]') as domain, COUNT(*) as n
    FROM commits
    WHERE committer_email LIKE '%%@%%'
    GROUP BY domain
    ORDER BY n DESC
""")

orgs = [o.strip('""') for o in orgs]

In [ ]:
percents = [(100 * c) / sum(counts) for c in counts if c / sum(counts) > 0.005]
if len(percents) < len(orgs):
    percents.append(100 - sum(percents))
    labels = orgs[:len(percents)-1] + ["others"]
else:
    labels = orgs

plt.figure(figsize=(10, 10))

plt.title("# contributions per organization")
_ = plt.pie(percents, labels=labels, textprops={"fontsize": 15})

In [ ]:
import numpy as np
from wordcloud import WordCloud

log_counts = np.log(counts) + 1
freqs = {org: log_counts[i] for (i, org) in enumerate(orgs)}

wordcloud = WordCloud(
    width=400,
    height=400,
    background_color="white",
    contour_color="blue",
    contour_width=1.0,
    colormap="viridis",
    min_font_size=4
)
wordcloud.generate_from_frequencies(freqs)
plt.figure(figsize=(7, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
top_contributors, counts = run("""
    SELECT committer_email, COUNT(*) as n
    FROM commits
    GROUP BY committer_email
    ORDER BY n DESC;
""")

In [ ]:
percents = [(100 * c) / sum(counts) for c in counts if c / sum(counts) > 0.003]
if len(percents) < len(top_contributors):
    percents.append(100 - sum(percents))
    labels = list(top_contributors[:len(percents)-1]) + ["others"]
else:
    labels = top_contributors
plt.figure(figsize=(10, 10))

plt.title("# contributions per email", pad=80.0)
_ = plt.pie(percents, labels=labels, rotatelabels=True, textprops={"fontsize": 15})

In [ ]:
run_and_print("""
    SELECT file_path,
           uast_extract(uast(blob_content,
                             LANGUAGE(file_path),
                             '//uast:Identifier'),
           "Name") name
    FROM blobs
         NATURAL JOIN commit_files
         NATURAL JOIN refs
    WHERE repository_id = 'c2spoon'
      AND LANGUAGE(file_path) = 'Java'
      AND ref_name = 'HEAD';
""")

In [ ]:
repos, last_commit_messages = run("""
    SELECT r.repository_id, c.commit_message
    FROM refs r
    NATURAL JOIN commits c
    WHERE r.ref_name = 'HEAD'
""")

In [ ]:
for repo, last_commit_message in zip(repos, last_commit_messages):
    display_html('<b>%s</b>: %s' % (repo, last_commit_message))